# Stationary scatterstates

In this notebook I will demonstrate how one can find the stationary scatterstate in a spin-1 heisenberg model. This should work for generic systems, as long as the excitations are topologically trivial.

In [1]:
using Revise,MPSKit, TensorKit, MPSKitExperimental, MPSKitModels, KrylovKit

## Step 1

always the same - we find the groundstate.

In [2]:
th = heisenberg_XXX(SU2Irrep);
ts = InfiniteMPS(th.pspaces,[Rep[SU₂](1//2=>20,3//2=>10,5//2=>5)]);
(ts,gs_env) = find_groundstate(ts,th,VUMPS(maxiter=200,verbose=false));

## Step 2

Step 2 is to find the assymptotically free part of the scatterstate. This determines the total energy of the scatterstate, as the sum of the energies of the free particles.

In [3]:
K = 0.0; # total momentum
dK = 0.3; # relative momentum

K1 = (-pi + K/2 + dK/2);
K2 = (pi + K/2 -dK/2);
(E1,A) = excitations(th,QuasiparticleAnsatz(),K1,ts,sector = SU2Irrep(1));
(E2,B) = excitations(th,QuasiparticleAnsatz(),K2,ts,sector = SU2Irrep(1));
E = E1[1]+E2[1];

A and B contain the left-gauged quasiparticles, and we can use this to make the two assymptotic parts of the scatterstate:

In [4]:
A_lg = A[1];
B_lg = B[1];

A_rg = convert(RightGaugedQP,A_lg);
B_rg = convert(RightGaugedQP,B_lg);


BA_asymptotic = AssymptoticScatter(B_lg,A_rg);
AB_asymptotic = AssymptoticScatter(A_lg,B_rg);

# Step 3

From step 3 onwards we will start to make use of momentum windows. The idea is that (H-E) acting on an assymptotic scatterstate should always be of the form of a momentum window. That will later on allow us to simplify some terms in our costfunction.

Step 3 itself is to find the window for which | window > = (H-E) | assymptotic > holds. This will only be approximate, and depending on the size and bond dimension of the window we can approximate this to a better degree.

In [6]:
len = 20; # size of the momentumwindow
windowchi = Rep[SU₂](1//2=>30,3//2=>20,5//2=>10,7//2=>5,9//2=>2,11//2=>1); # bond dimension of the momentumwindow

BA_projected = LeftGaugedMW(rand,len,windowchi,ts,momentum = K1 + K2+0im,utilspace= MPSKit.utilleg(BA_asymptotic));    
AB_projected = LeftGaugedMW(rand,len,windowchi,ts,momentum = K1 + K2+0im,utilspace= MPSKit.utilleg(BA_projected));

begin
    (AB_projected,_) = approximate(AB_projected,(th,AB_asymptotic),DMRG(tol=1e-12,maxiter=20,verbose=true));
    (BA_projected,_) = approximate(BA_projected,(th,BA_asymptotic),DMRG(tol=1e-12,maxiter=20,verbose=true));
end;

┌ Info: dmrg iter 0 error 85.31556538452868
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


┌ Info: dmrg iter 1 error 0.040185063558800554
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


┌ Info: dmrg iter 2 error 0.003301447675325622
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


┌ Info: dmrg iter 3 error 0.0016669503020225087
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


┌ Info: dmrg iter 4 error 0.0011430849047240957
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


...

┌ Info: dmrg iter 15 error 8.401382895422423e-5
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


┌ Info: dmrg iter 16 error 7.023425325310239e-5
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


┌ Info: dmrg iter 17 error 5.871629727898857e-5
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


┌ Info: dmrg iter 18 error 4.9084593275224927e-5
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289


┌ Info: dmrg iter 19 error 4.10297118104067e-5
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:289
┌ Warning: dmrg failed to converge 4.10297118104067e-5>1.0e-12
└ @ MPSKitExperimental /home/maarten/projects/MPSKitExperimental.jl/src/momentumwindow/assymptotic.jl:295


## Step 4 

In the last step we will attempt to solve

(H-E) [ | window > + | AB_assymptotic > + S | BA_assymptotic > ] = 0

using the windows from step 3, we can simplify this to

(H - E) | window > + | AB_projected > + S | BA_projected > = 0

This is reminiscent of a correction vector problem, and it seems that these can be solved by finding:

P = projector on the tangent space of window

P (H - E) | window > + P | AB_projected > + P S | BA_projected > = 0

< BA_projected | (H - E) | window > + < BA_projected | AB_projected > + < BA_projected | S | BA_projected > = 0

The last equation is interesting, as | BA_projected > is equal to (H-E) | BA_assymptotic>. It seems natural to remove this (H-E) factor, and the final set of equations are then numerically easier to solve!

=> < BA_assymptotic | (H - E) | window > + < BA_assymptotic | AB_projected > + < BA_assymptotic | S | BA_projected > = 0

You can do this using for example a sweeping scheme :

In [7]:

# you can tune both the bond dimension and the length of this window independently, but it doesn't appear to matter that much
B = LeftGaugedMW(rand,1,windowchi,ts,momentum=K1+K2+0im,utilspace=MPSKit.utilleg(AB_projected));

# some initial guess, which is always completely wrong
S = -inv(partialdot(BA_projected,BA_projected))*partialdot(BA_projected,AB_projected)

# environments for < B | H | B >
envs = environments(B,(th,B),gs_env,gs_env);

for topit = 1:5 # in principle you need to sweep until converged
    for row in 1:size(B.AC,1),col in 1:size(B.AC,2)
        y_1 = -projdown(row,col,AB_projected,B);
        y_2 = -partialdot(BA_asymptotic,AB_projected);

        x_1 = B.AC[row,col];
        x_2 = S;

        (sol,convhist) = linsolve(RecursiveVec((y_1,y_2)),RecursiveVec((x_1,x_2)),GMRES(tol=1e-9)) do x
            (c_B,c_S) = x.vecs;
            B.AC[row,col] = c_B;
            envs = environments(B,(th,B),gs_env,gs_env);
            
            SBA_projected = lmul!(c_S,copy(BA_projected))

            v_1 = MPSKit.ac_proj(row,col,B,envs)-E*c_B+projdown(row,col,SBA_projected,B);
            v_2 = partialdot(BA_projected,B)+partialdot(BA_asymptotic,SBA_projected);
            RecursiveVec((v_1,v_2))
        end
        
        B.AC[row,col] = sol[1];
        S = sol[2];

        @show norm(S-x_2),norm(B.AC[row,col]-x_1) # convergence metrics
        @show convhist.converged,convhist.normres

        @show S
        flush(stdout)
    end;
end

(norm(S - x_2), norm(B.AC[row, col] - x_1)) = (1.632772098222669, 8.47109847540111)
(convhist.converged, convhist.normres) = (1, 9.609325193027973e-10)


S = TensorMap(Rep[SU₂](0=>1, 1=>1, 2=>1) ← Rep[SU₂](0=>1, 1=>1, 2=>1)):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
 -0.8660484276734622 + 0.49488856210683807im
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()):
 -0.45084852924939817 - 0.8931813653215079im
* Data for fusiontree FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()) ← FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()):
 -0.8476840111580839 - 0.5303904536388452im



(norm(S - x_2), norm(B.AC[row, col] - x_1)) = (0.0, 0.0)
(convhist.converged, convhist.normres) = (1, 9.609325193027973e-10)
S = TensorMap(Rep[SU₂](0=>1, 1=>1, 2=>1) ← Rep[SU₂](0=>1, 1=>1, 2=>1)):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
 -0.8660484276734622 + 0.49488856210683807im
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()):
 -0.45084852924939817 - 0.8931813653215079im
* Data for fusiontree FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()) ← FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()):
 -0.8476840111580839 - 0.5303904536388452im



(norm(S - x_2), norm(B.AC[row, col] - x_1)) = (0.0, 0.0)
(convhist.converged, convhist.normres) = (1, 9.609325193027973e-10)
S = TensorMap(Rep[SU₂](0=>1, 1=>1, 2=>1) ← Rep[SU₂](0=>1, 1=>1, 2=>1)):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
 -0.8660484276734622 + 0.49488856210683807im
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()):
 -0.45084852924939817 - 0.8931813653215079im
* Data for fusiontree FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()) ← FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()):
 -0.8476840111580839 - 0.5303904536388452im



(norm(S - x_2), norm(B.AC[row, col] - x_1)) = (0.0, 0.0)
(convhist.converged, convhist.normres) = (1, 9.609325193027973e-10)
S = TensorMap(Rep[SU₂](0=>1, 1=>1, 2=>1) ← Rep[SU₂](0=>1, 1=>1, 2=>1)):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
 -0.8660484276734622 + 0.49488856210683807im
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()):
 -0.45084852924939817 - 0.8931813653215079im
* Data for fusiontree FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()) ← FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()):
 -0.8476840111580839 - 0.5303904536388452im



(norm(S - x_2), norm(B.AC[row, col] - x_1)) = (0.0, 0.0)
(convhist.converged, convhist.normres) = (1, 9.609325193027973e-10)
S = TensorMap(Rep[SU₂](0=>1, 1=>1, 2=>1) ← Rep[SU₂](0=>1, 1=>1, 2=>1)):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
 -0.8660484276734622 + 0.49488856210683807im
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()):
 -0.45084852924939817 - 0.8931813653215079im
* Data for fusiontree FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()) ← FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()):
 -0.8476840111580839 - 0.5303904536388452im



## What now?

We obtain the S matrix, but the code also seems to work for complex momenta! We can use the same code to calculate dynamical structure factors. 

There doesn't seem to be an inherent obstacle in going to a regime where multiple two-particle states can contribute to the scatterprocess, maybe that could be interesting?

The same approach should work for quasiparticle scattering on an impurity in the spin chain, and it should be far easier to implement.